<a href="https://colab.research.google.com/github/TJSun009/University-Projects/blob/main/Chord_Classifier_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Fast.ai Libraries

In [54]:
!pip install -Uqq fastai
from fastcore.all import *
from fastai.vision.all import *
from fastai.vision.widgets import *

## Import Voila

In [64]:
!pip install voila
!jupyter serverextension enable --sys-prefix voila

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.6 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 71 kB 188 kB/s 
     |████████████████████████████████| 346 kB 57.3 MB/s 
     |████████████████████████████████| 106 kB 59.3 MB/s 
     |████████████████████████████████| 273 kB 67.0 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 80 kB 8.8 MB/s 
     |████████████████████████████████| 423 kB 58.4 MB/s 
     |████████████████████████████████| 60 kB 6.4 MB/s 
     |████████████████████████████████| 133 kB 57.5 MB/s 
     |████████████████████████████████| 107 kB 58.1 MB/s 
     |████████████████████████████████| 86 kB 3.8 MB/s 
  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.1.1
    Uninstalling traitlets-5.1.1:
      Successfully uninstalled traitlets-5.

Enabling: voila
- Writing config: /usr/etc/jupyter
    - Validating...
      voila 0.4.0 OK


## Audio to Image Transform



In [56]:
import matplotlib.pyplot as plt
import numpy as np
import wave
import librosa
import librosa.display

def save_mel_spectogram(audio_file, dest=''):
    # Adapted from https://towardsdatascience.com/audio-deep-learning-made-simple-part-2-why-mel-spectrograms-perform-better-aad889a93505

    samples, sample_rate = librosa.load(audio_file, sr=None)

    # std spectogram
    sgram = librosa.stft(samples)

    # use the mel-scale instead of raw frequency
    sgram_mag, _ = librosa.magphase(sgram)
    mel_scale_sgram = librosa.feature.melspectrogram(S=sgram_mag, sr=sample_rate)
    
    # use the decibel scale to get the final Mel Spectrogram
    mel_sgram = librosa.amplitude_to_db(mel_scale_sgram, ref=np.min)
    librosa.display.specshow(mel_sgram, sr=sample_rate)
    
    plt.axis('off')

    path = f"{(dest if dest else audio_file).replace('.wav', '.png')}"
    plt.savefig(path)
    plt.close()
    return path

## Import Model

In [57]:
path = Path("/content/drive/MyDrive/Year 3/Dissertation/Projects/Training/Fast.ai/Lesson_1/")
learn_inf = load_learner(path/'export.pkl')

## Model Notebook App

### Widgets

In [61]:
btn_upload = widgets.FileUpload(accept=".wav")
btn_run = widgets.Button(description="Classify")
lbl_pred = widgets.Label()
out_pl = widgets.Output()
out_so = widgets.Audio(format="wav")

### Upload

In [62]:
def classify_chord(change):

  path = Path("/content/")

  fname = btn_upload.metadata[0]['name']

  # print(fname)
  # print(btn_upload.value[fname]["content"])
  # print(btn_upload.content)

  content = btn_upload.value[fname]["content"]

  with open(f"{path}/{fname}", mode='bx') as file:
    file.write(content)

  spectrogram = save_mel_spectogram(f"{path}/{fname}")

  img = PILImage.create(spectrogram)
  out_pl.clear_output()
  out_so.set_value_from_file(f"{path}/{fname}")
  with out_pl: display(img.to_thumb(128,128))
  pred,pred_idx,probs = learn_inf.predict(img)
  lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

In [63]:
btn_run.on_click(classify_chord)
VBox([widgets.Label('Upload your Chord .wav file'), btn_upload, btn_run, out_pl, out_so, lbl_pred])